In [2]:
import pandas as pd
import requests
import random 
from bs4 import BeautifulSoup

In [ ]:
response = requests.get("https://d23.com/list-of-disney-films/")
html_string = response.text
document = BeautifulSoup(html_string, "html.parser")
document

In [4]:
list = document.find_all("span", style="font-size: 20px;")
movies_messy = [movie.text for movie in list]
movies = [item for sublist in movies_messy for item in sublist.split('\n')] # split items with multiple movies
movies = [i for i in movies if i] # remove empty strings 
print(movies)

['1. 1937:\xa0Snow White and the Seven Dwarfs (G)', '2. 1940:\xa0Pinocchio (G)', '3. 1940:\xa0Fantasia (G)', '4. 1941:\xa0The Reluctant Dragon', '5. 1941:\xa0Dumbo (G)', '6. 1942:\xa0Bambi (G)', '7. 1943:\xa0Saludos Amigos', '8. 1943:\xa0Victory Through Air Power', '9. 1945:\xa0The Three Caballeros (G)', '10. 1946:\xa0Make Mine Music', '11. 1946:\xa0Song of the South (G)', '12. 1947:\xa0Fun and Fancy Free', '13. 1948:\xa0Melody Time', '14. 1949:\xa0So Dear to My Heart (G)', '15. 1949:\xa0The Adventures of Ichabod and Mr. Toad (G)', '16. 1950:\xa0Cinderella (G)', '17. 1950:\xa0Treasure Island (PG)', '18. 1951:\xa0Alice in Wonderland (G)', '19. 1952:\xa0The Story of Robin Hood and His Merrie Men (PG)', '20. 1953:\xa0Peter Pan (G)', '21. 1953:\xa0The Sword and the Rose (PG)', '22. 1953:\xa0The Living Desert', '23. 1954:\xa0Rob Roy, the Highland Rogue', '24. 1954:\xa0The Vanishing Prairie', '25. 1954:\xa020,000 Leagues Under the Sea (G)', '26. 1955:\xa0Davy Crockett, King of the Wild Front

In [5]:
import re

def extract_data(movie):
    movie = movie.replace('[Disney+]', '')
    year = re.search(r'\b\d{4}\b', movie).group()
    try:
        title = re.search(r':(.*?) \(', movie).group(1)
        rating = re.search(r'\(([^)]+)\)$', movie).group(1)
    except:
        print(movie, "no rating")
        title = re.search(r':(.*)', movie).group(1)
        rating = 'NaN'
    try:
        branch = re.search(r'\(([^()]+)\)[^()]*\(([^()]+)\)', movie).group(1)
    except:
        branch = 'Standard'

    return({'Title': title.strip(), 'Year': year, 'Rating': rating, 'Branch': branch})

# extract_data(movies[795])

{'Title': 'Thor: Love and Thunder',
 'Year': '2022',
 'Rating': 'PG-13',
 'Branch': 'Marvel'}

In [6]:
disney_df = pd.DataFrame(columns = ['Title', 'Year', 'Rating', 'Branch'])

for movie in movies:
    new_row = extract_data(movie)
    print(new_row)
    disney_df.loc[len(disney_df)] = new_row

disney_df

{'Title': 'Snow White and the Seven Dwarfs', 'Year': '1937', 'Rating': 'G', 'Branch': 'Standard'}
{'Title': 'Pinocchio', 'Year': '1940', 'Rating': 'G', 'Branch': 'Standard'}
{'Title': 'Fantasia', 'Year': '1940', 'Rating': 'G', 'Branch': 'Standard'}
4. 1941: The Reluctant Dragon no rating
{'Title': 'The Reluctant Dragon', 'Year': '1941', 'Rating': 'NaN', 'Branch': 'Standard'}
{'Title': 'Dumbo', 'Year': '1941', 'Rating': 'G', 'Branch': 'Standard'}
{'Title': 'Bambi', 'Year': '1942', 'Rating': 'G', 'Branch': 'Standard'}
7. 1943: Saludos Amigos no rating
{'Title': 'Saludos Amigos', 'Year': '1943', 'Rating': 'NaN', 'Branch': 'Standard'}
8. 1943: Victory Through Air Power no rating
{'Title': 'Victory Through Air Power', 'Year': '1943', 'Rating': 'NaN', 'Branch': 'Standard'}
{'Title': 'The Three Caballeros', 'Year': '1945', 'Rating': 'G', 'Branch': 'Standard'}
10. 1946: Make Mine Music no rating
{'Title': 'Make Mine Music', 'Year': '1946', 'Rating': 'NaN', 'Branch': 'Standard'}
{'Title': 'Song

,Title,Year,Rating,Branch
0,Snow White and the Seven Dwarfs,1937,G,Standard
1,Pinocchio,1940,G,Standard
2,Fantasia,1940,G,Standard
3,The Reluctant Dragon,1941,NaN,Standard
4,Dumbo,1941,G,Standard
...,...,...,...,...
807,Guardians of the Galaxy Vol. 3,2023,PG-13,Marvel
808,The Little Mermaid,2023,PG,Standard
809,Elemental,2023,PG,Pixar
810,Indiana Jones and the Dial of Destiny,2023,PG-13,Lucasfilm


In [8]:
disney_df.to_csv('../Disney/disney_movies.csv', encoding='utf-8', index=True)